# Capstone 3 Data Wrangling- Jessica Williams

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
import spacy
from yellowbrick.text import FreqDistVisualizer
from pathlib import Path
nlp=spacy.load('en_core_web_sm')
from textblob import TextBlob, Word, Blobber

from sb_utils import save_file

Lets take a look at both dataframes.

In [2]:
wine_list = pd.read_csv('C:/Users/jwatki8/Downloads/XWines_Full_100K_wines.csv')

In [3]:
wine_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100646 entries, 0 to 100645
Data columns (total 17 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   WineID      100646 non-null  int64  
 1   WineName    100646 non-null  object 
 2   Type        100646 non-null  object 
 3   Elaborate   100646 non-null  object 
 4   Grapes      100646 non-null  object 
 5   Harmonize   100646 non-null  object 
 6   ABV         100646 non-null  float64
 7   Body        100646 non-null  object 
 8   Acidity     100646 non-null  object 
 9   Code        100646 non-null  object 
 10  Country     100646 non-null  object 
 11  RegionID    100646 non-null  int64  
 12  RegionName  100646 non-null  object 
 13  WineryID    100646 non-null  int64  
 14  WineryName  100646 non-null  object 
 15  Website     82779 non-null   object 
 16  Vintages    100646 non-null  object 
dtypes: float64(1), int64(3), object(13)
memory usage: 13.1+ MB


In [4]:
wine_list.head()

,WineID,WineName,Type,Elaborate,Grapes,Harmonize,ABV,Body,Acidity,Code,Country,RegionID,RegionName,WineryID,WineryName,Website,Vintages
0,100001,Espumante Moscatel,Sparkling,Varietal/100%,['Muscat/Moscato'],"['Pork', 'Rich Fish', 'Shellfish']",7.5,Medium-bodied,High,BR,Brazil,1001,Serra Gaúcha,10001,Casa Perini,http://www.vinicolaperini.com.br,"[2020, 2019, 2018, 2017, 2016, 2015, 2014, 201..."
1,100002,Ancellotta,Red,Varietal/100%,['Ancellotta'],"['Beef', 'Barbecue', 'Codfish', 'Pasta', 'Pizz...",12.0,Medium-bodied,Medium,BR,Brazil,1001,Serra Gaúcha,10001,Casa Perini,http://www.vinicolaperini.com.br,"[2016, 2015, 2014, 2013, 2012, 2011, 2010, 200..."
2,100003,Cabernet Sauvignon,Red,Varietal/100%,['Cabernet Sauvignon'],"['Beef', 'Lamb', 'Poultry']",12.0,Full-bodied,High,BR,Brazil,1001,Serra Gaúcha,10002,Castellamare,https://www.emporiocastellamare.com.br,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201..."
3,100004,Virtus Moscato,White,Varietal/100%,['Muscat/Moscato'],['Sweet Dessert'],12.0,Medium-bodied,Medium,BR,Brazil,1001,Serra Gaúcha,10003,Monte Paschoal,http://www.montepaschoal.com.br,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201..."
4,100005,Maison de Ville Cabernet-Merlot,Red,Assemblage/Bordeaux Red Blend,"['Cabernet Sauvignon', 'Merlot']","['Beef', 'Lamb', 'Game Meat', 'Poultry']",11.0,Full-bodied,Medium,BR,Brazil,1001,Serra Gaúcha,10000,Aurora,http://www.vinicolaaurora.com.br,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201..."


In [5]:
wine_ratings = pd.read_csv('C:/Users/jwatki8/Downloads/XWines_Full_21M_ratings.csv')

C:\Users\jwatki8\AppData\Local\Temp\ipykernel_16648\3750174202.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  wine_ratings = pd.read_csv('C:/Users/jwatki8/Downloads/XWines_Full_21M_ratings.csv')


In [6]:
wine_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21013536 entries, 0 to 21013535
Data columns (total 6 columns):
 #   Column    Dtype  
---  ------    -----  
 0   RatingID  int64  
 1   UserID    int64  
 2   WineID    int64  
 3   Vintage   object 
 4   Rating    float64
 5   Date      object 
dtypes: float64(1), int64(3), object(2)
memory usage: 961.9+ MB


In [7]:
wine_ratings.head()

,RatingID,UserID,WineID,Vintage,Rating,Date
0,1,1604441,136103,1950,4.0,2019-10-14 11:20:52
1,2,1291483,136103,1950,5.0,2019-11-28 03:36:33
2,3,1070605,104036,1950,5.0,2017-12-28 10:15:55
3,4,1080181,144864,1950,5.0,2016-06-23 02:16:22
4,5,1834379,111430,1950,5.0,2021-05-16 17:58:14


In [8]:
wine_list.shape

(100646, 17)

In [9]:
wine_list.dtypes

WineID          int64
WineName       object
Type           object
Elaborate      object
Grapes         object
Harmonize      object
ABV           float64
Body           object
Acidity        object
Code           object
Country        object
RegionID        int64
RegionName     object
WineryID        int64
WineryName     object
Website        object
Vintages       object
dtype: object

The first thing I would like to do with the wine list dataframe is drop the website column as I don’t see this as being of any use.

In [10]:
wine_list=wine_list.drop(columns='Website')

In [11]:
wine_list.dtypes

WineID          int64
WineName       object
Type           object
Elaborate      object
Grapes         object
Harmonize      object
ABV           float64
Body           object
Acidity        object
Code           object
Country        object
RegionID        int64
RegionName     object
WineryID        int64
WineryName     object
Vintages       object
dtype: object

Let’s check for null values

In [12]:
wine_list.isnull().sum()

WineID        0
WineName      0
Type          0
Elaborate     0
Grapes        0
Harmonize     0
ABV           0
Body          0
Acidity       0
Code          0
Country       0
RegionID      0
RegionName    0
WineryID      0
WineryName    0
Vintages      0
dtype: int64

Now I want to examine the unique values in the different string attribute columns.

In [13]:
wine_list['Type'].value_counts()

Red             56162
White           29196
Sparkling        7366
Rosé             4401
Dessert          2409
Dessert/Port     1112
Name: Type, dtype: int64

In [14]:
wine_list['Elaborate'].value_counts()

Varietal/100%                        72496
Assemblage/Blend                     19257
Assemblage/Bordeaux Red Blend         4374
Assemblage/Rhône Red Blend            1254
Assemblage/Valpolicella Red Blend      803
Assemblage/Champagne Blend             699
Assemblage/Portuguese Red Blend        562
Assemblage/Port Blend                  442
Assemblage/Provence Rosé Blend         258
Assemblage/Meritage Red Blend          104
Assemblage/Portuguese White Blend       78
Varietal/>75%                           68
Assemblage/Bourgogne Red Blend          67
Assemblage/Rioja Red Blend              60
Assemblage/Cava Blend                   42
Assemblage/Tuscan Red Blend             26
Assemblage/Priorat Red Blend            15
Assemblage/Soave White Blend            14
Assemblage/Chianti Red Blend            11
Assemblage/Meritage White Blend          6
Assemblage/Rioja White Blend             5
Assemblage/Bourgogne White Blend         5
Name: Elaborate, dtype: int64

In [15]:
wine_list['Grapes'].value_counts()

['Chardonnay']                                                                                                                                8702
['Pinot Noir']                                                                                                                                7892
['Cabernet Sauvignon']                                                                                                                        5532
['Syrah/Shiraz']                                                                                                                              3864
['Riesling']                                                                                                                                  3641
                                                                                                                                              ... 
['Sangiovese', 'Primitivo', 'Negroamaro']                                                                             

In [16]:
wine_list['Harmonize'].value_counts()

['Beef', 'Lamb', 'Game Meat', 'Poultry']                     9575
['Beef', 'Veal', 'Game Meat', 'Poultry']                     8548
['Pork', 'Rich Fish', 'Vegetarian', 'Poultry']               4538
['Beef', 'Pasta', 'Lamb', 'Game Meat']                       4522
['Beef', 'Lamb', 'Poultry']                                  4039
                                                             ... 
['Pasta', 'Fruit', 'Chocolate', 'Lean Fish']                    1
['Cheese', 'Spiced Fruit Cake', 'Fruit Dessert', 'Meat']        1
['Beef', 'Maturated Cheese', 'Sweet Dessert']                   1
['Shellfish', 'Pork', 'Appetizer', 'Snack', 'Cured Meat']       1
['Fish', 'Seafood']                                             1
Name: Harmonize, Length: 900, dtype: int64

In [17]:
wine_list['Body'].value_counts()

Full-bodied          43881
Medium-bodied        34397
Very full-bodied     10879
Light-bodied         10034
Very light-bodied     1455
Name: Body, dtype: int64

In [18]:
wine_list['Acidity'].value_counts()

High      79394
Medium    17842
Low        3410
Name: Acidity, dtype: int64

Next let’s check the wine Id variable for duplicates.

In [19]:
wine_list['WineID'].value_counts()

100001    1
167210    1
167208    1
167207    1
167206    1
         ..
133568    1
133567    1
133566    1
133565    1
200795    1
Name: WineID, Length: 100646, dtype: int64

In [20]:
wine_list['WineID'].value_counts().max()

1

It looks like the ids for the recipes are all unique.

Let’s examine the wine rating dataframe a little closer.

In [21]:
wine_ratings.dtypes

RatingID      int64
UserID        int64
WineID        int64
Vintage      object
Rating      float64
Date         object
dtype: object

Once again we will check for null values.

In [22]:
wine_ratings.isnull().sum()

RatingID    0
UserID      0
WineID      0
Vintage     0
Rating      0
Date        0
dtype: int64

In [23]:
wine_ratings['Vintage'].value_counts()

2016    2281547
2015    2263576
2017    2099372
2018    1943550
2014    1897335
         ...   
1950        171
1952        164
1956         62
1954         55
1951         30
Name: Vintage, Length: 74, dtype: int64

Now let’s take a look at the value range for the ratings columns in this dataframe.

In [24]:
print(wine_ratings['Rating'].min())
print(wine_ratings['Rating'].max())

1.0
5.0


In [25]:
wine_ratings['WineID'].value_counts()

155289    27415
179010    23626
179011    21216
111391    20913
167418    20817
          ...  
200675        5
200771        5
191650        5
111109        5
154991        5
Name: WineID, Length: 100646, dtype: int64

I would like to get the mean of the rating column grouped by Wine ID. In other words find the mean of the ratings for each recipe and then apply it to the recipe data frame by id#. Looking at the value counts for the Wine ID column in both data frames I can see that they contain the same number of unique values in each column. This mean that there is at least some rating data for all recipe ids. Now let’s calculate the mean rating.

In [26]:
wine_ratings['rating_mean'] = wine_ratings['Rating'].groupby(wine_ratings['WineID']).transform('mean')

In [27]:
wine_ratings.head()

,RatingID,UserID,WineID,Vintage,Rating,Date,rating_mean
0,1,1604441,136103,1950,4.0,2019-10-14 11:20:52,3.611725
1,2,1291483,136103,1950,5.0,2019-11-28 03:36:33,3.611725
2,3,1070605,104036,1950,5.0,2017-12-28 10:15:55,4.551546
3,4,1080181,144864,1950,5.0,2016-06-23 02:16:22,3.746667
4,5,1834379,111430,1950,5.0,2021-05-16 17:58:14,4.773632


In [28]:
wine_ratings.tail()

,RatingID,UserID,WineID,Vintage,Rating,Date,rating_mean
21013531,21013532,2015383,113302,N.V.,3.0,2019-02-16 14:15:48,3.778169
21013532,21013533,1868739,111440,N.V.,2.0,2018-09-30 16:47:05,4.203804
21013533,21013534,1402947,142467,N.V.,3.0,2021-01-29 19:21:14,3.633094
21013534,21013535,1360350,111440,N.V.,4.0,2021-07-26 14:02:14,4.203804
21013535,21013536,1192603,111393,N.V.,5.0,2016-11-17 04:48:43,4.261586


Let’s look at the minimum and maximum values for our new rating mean column.

In [29]:
print(wine_ratings['rating_mean'].min())
print(wine_ratings['rating_mean'].max())


1.25
5.0


Next let’s take a look at some of the user information. Let’s see how many unique raters we have out of all of our ratings.

In [30]:
wine_ratings['UserID'].nunique()

1056079

Our dataset contains 1,056,079 unique raters. Let’s examine further.

In [31]:
print(wine_ratings['UserID'].value_counts().max())

2986


In [32]:
wine_ratings['UserID'].value_counts().head(20)

1084433    2986
1034989    2979
1070878    2613
1048267    2597
1160536    2392
1000272    2309
1006657    2307
1019610    2291
1107271    2225
1035125    2101
1163129    2059
1165081    2004
1169598    1995
1165941    1923
1008661    1909
1028358    1859
1089420    1804
1878271    1800
1217756    1787
1151866    1721
Name: UserID, dtype: int64

In [33]:
wine_ratings['UserID'].value_counts().tail(20)

2007445    5
1376871    5
1831028    5
1161762    5
1524599    5
1331122    5
2028527    5
1921392    5
1347527    5
1415002    5
1912519    5
1721488    5
1495745    5
2008918    5
1010653    5
1808349    5
1301644    5
1344498    5
1407463    5
1064572    5
Name: UserID, dtype: int64

It looks like the maximum number of times someone reviewed a wine was 2986 while the minimum number of times someone rated wine was 5. This means that we have no one time raters in this dataset and a good amount of high frequency raters. 

Now we are dealing with two fairly clean dataframes. We dropped a few items and created some new variables. I think it is too early to combine the two data frames as I would like to examine more relationships and correlations during EDA that may lead to more variables being created.

In [32]:
datapath ='C:/Users/jwatki8/Documents/Data Science Program information/My Capstone project'
save_file(wine_list,'wine_list.csv', datapath)

Writing file.  "C:/Users/jwatki8/Documents/Data Science Program information/My Capstone project\wine_list.csv"


In [33]:
datapath ='C:/Users/jwatki8/Documents/Data Science Program information/My Capstone project'
save_file(wine_ratings,'wine_ratings.csv', datapath)

Writing file.  "C:/Users/jwatki8/Documents/Data Science Program information/My Capstone project\wine_ratings.csv"
